In [1]:
import pandas as pd
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cbpro
from pathlib import Path
import plotly.express as px

public_client = cbpro.PublicClient()


In [2]:
class MinerMeta(type):
    def compile_historic(self, timestamp=False, write_csv=False, read_csv=False):
        file = Path.cwd() / f"{self.coin}_histdata.json"
        if read_csv == True:
            df = pd.read_csv(file, index_col="time")
            return df
        else:
            start_date = datetime(2020, 1, 1)
            end_date = datetime.today()
            delta = timedelta(hours=300)
            df = pd.DataFrame()

            while start_date <= end_date:
                historic = public_client.get_product_historic_rates(
                    f"{self.coin}-USD",
                    granularity=3600,
                    start=start_date,
                    end=start_date + delta,
                )
                start_date += delta
                df = df.append(historic, ignore_index=True)
            df.columns = ["time", "low", "high", "open", "close", "volume"]
            date_time = pd.to_datetime(df["time"], unit="s")

            timestamp_s = date_time.map(pd.Timestamp.timestamp)
            day = 24 * 60 * 60
            year = (365.2425) * day
            df["Day sin"] = np.sin(timestamp_s * (2 * np.pi / day))
            df["Day cos"] = np.cos(timestamp_s * (2 * np.pi / day))
            df["Year sin"] = np.sin(timestamp_s * (2 * np.pi / year))
            df["Year cos"] = np.cos(timestamp_s * (2 * np.pi / year))
            df.reset_index(drop=True, inplace=True)

            if timestamp is True:
                pass
            else:
                df["time"] = pd.to_datetime(df["time"], unit="s")

            if write_csv is True:
                df.to_csv(file, index=False)
        # df["time"] = pd.to_datetime(df["time"], unit="s")
        # df["time"] = date_time.map(pd.Timestamp.timestamp)

        # data = np.array(df, dtype=np.float32)
        # ds = tf.keras.preprocessing.timeseries_dataset_from_array(
        #     data=df,
        #     targets="close",
        #     sequence_length=10,
        #     sequence_stride=1,
        #     shuffle=False,
        #     batch_size=64,
        # )

        return df

    def plot_cols(self, df, cols=None):
        if cols == None:
            plot_features = df[list(df.columns)]
        else:
            plot_features = df[cols]

        index = df["time"]
        fig, axs = plt.subplots(len(plot_features))

        for x in len(plot_features):
            axs[x].plot(df["time"], plot_features[x])
        # fft = tf.signal.rfft(df[cols])

    def do_fft(self):
        df = self.compile_historic()
        today = datetime.today()

        fft = tf.signal.rfft(df["close"])
        f_per_dataset = np.arange(0, len(fft))

        n_samples_h = len(df["close"])
        hours_per_year = 24 * 365.2524
        years_per_dataset = n_samples_h / (hours_per_year)

        f_per_year = f_per_dataset / years_per_dataset
        plt.step(f_per_year, np.abs(fft))
        plt.xscale("log")
        plt.ylim(0, 400000)
        plt.xlim([0.1, max(plt.xlim())])
        plt.xticks([1, 365.2524], labels=["1/Year", "1/day"])
        _ = plt.xlabel("Frequency (log scale)")

        return plt.show()

    def __call__(self, *args, **kwargs):

        cls = type.__call__(self, *args)

        setattr(cls, "compile_historic", self.compile_historic)
        setattr(cls, "plot_cols", self.plot_cols)
        setattr(cls, "do_fft", self.do_fft)
        # setattr(cls, "make_dataset", self.compile_historic)

        # for key, value in historic.items():
        #     setattr(cls, "hist_" + key, value)
        # for key, value in ticker.items():
        #     setattr(cls, "tick_" + key, value)

        return cls


class eth(metaclass=MinerMeta):
    coin = "eth"


class btc(metaclass=MinerMeta):
    coin = "btc"


In [3]:
df = eth.compile_historic(read_csv=True)

today = datetime.today()

df = df[0:200]

cols = list(df.columns)
df = df[cols[:2]]
px.scatter(df)
